In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,VotingClassifier, AdaBoostClassifier,GradientBoostingClassifier


# Import Data

In [2]:
teams = pd.read_csv('Teams.csv')   # (362, 2)  Team_Id; Team_Name
seasons = pd.read_csv('Seasons.csv')  # (33, 6)  
r_compact = pd.read_csv('RegularSeasonCompactResults.csv')   # (150684, 8)
r_detail = pd.read_csv('RegularSeasonDetailedResults.csv')   # (76636, 34)
t_compact = pd.read_csv('TourneyCompactResults.csv')   # (2050, 8)
t_detail = pd.read_csv('TourneyDetailedResults.csv')   # (914, 34)
seed = pd.read_csv('TourneySeeds.csv')   # (2150, 3)
slot = pd.read_csv('TourneySlots.csv')   # (2117, 4)

In [4]:
# function to check missing value
def missing(df):
    missing_value = df[df.columns[df.isnull().any()]].isnull().sum()
    return missing_value

# Create new dataset contains every team's regular season's statistics

In [3]:
# pick the columns related with winning team
rw_detail = r_detail[['Season', 'Daynum', 'Wteam', 'Wscore',  'Wloc',
       'Numot', 'Wfgm', 'Wfga', 'Wfgm3', 'Wfga3', 'Wftm', 'Wfta', 'Wor', 'Wdr',
       'Wast', 'Wto', 'Wstl', 'Wblk', 'Wpf']].copy()

# the result for winning team is 1
rw_detail['result'] = 1   # 1 for win, 0 for lose

# remove the 'W' for column name
rw_detail.columns = [i.replace('W','') for i in rw_detail.columns]

In [4]:
# Do the same thing for loss team
rl_detail = r_detail[['Season','Daynum','Lteam','Wloc',
       'Numot','Lscore','Lfgm', 'Lfga', 'Lfgm3', 'Lfga3',
       'Lftm', 'Lfta', 'Lor', 'Ldr', 'Last', 'Lto', 'Lstl', 'Lblk', 'Lpf']].copy()

rl_detail['result'] = 0

rl_detail.columns = [i.replace('L','') for i in rl_detail.columns]

rl_detail_loc_dict = {'H':'A', 'A':'H', 'N':'N'}

rl_detail['Wloc'] = rl_detail['Wloc'].map(rl_detail_loc_dict)

rl_detail.columns = [i.replace('W','') for i in rl_detail.columns]

In [5]:
# concat win team and loss team
rwl_detail = pd.concat([rw_detail, rl_detail])   #  (153272, 20)

In [6]:
# create three new features by calculating the percentage
rwl_detail['fgp'] = rwl_detail['fgm']/rwl_detail['fga']
rwl_detail['fgp3'] = rwl_detail['fgm3']/rwl_detail['fga3']
rwl_detail['ftp'] = rwl_detail['ftm']/rwl_detail['fta']

In [7]:
# pick the features which have higher correlation with result
result_corr = rwl_detail.corr()[['result']]
high_result_corr = result_corr[abs(result_corr['result']) > 0.05].drop(['result'])   # (16, 1)

In [8]:
# Create the DataFrame with contain Season, team, and regular season's statistic
rwl_detail = rwl_detail[list(high_result_corr.index) + ['team','Season']]

In [9]:
# Calulate avarage stats for each season each team 
# Drop the season 2017 for laterly create training dataset
r_detail_mean = rwl_detail.groupby(['Season' ,'team']).mean().drop([2017]).reset_index()

# Create dataset contains number of winning games of last tournament for every team

In [21]:
t_compact = t_compact[['Season','Wteam','Lteam']]

# create dataframe for tourney number games win
t_win_games = t_compact[['Season','Wteam','Lteam']]
t_win_games['Season'] = t_win_games['Season'] + 1
t_win_games['number_win'] = 1

t_win_games = t_win_games.groupby(['Season','Wteam']).count().reset_index().drop(['Lteam'],axis=1)

In [22]:
# merge number win games t for win team
t_compact = pd.merge(left=t_compact, right=t_win_games, how='left', on=['Season', 'Wteam'])

# change column name
t_win_games_loss = t_win_games.rename(columns={'Wteam':'Lteam'})
# merge number win games t for loss team
t_compact = pd.merge(left=t_compact, right=t_win_games_loss, how='left', on=['Season', 'Lteam'])

# create new column for different number of t winning between win team and loss team
t_compact['t_number_win_diff'] = t_compact['number_win_x'] - t_compact['number_win_y']

t_compact = t_compact.drop(['number_win_x','number_win_y'], axis=1)

# Create dataset contains seed number for each team

In [23]:

# change feature Seed type to integer
seed['numeric_seed'] = [int(i[1:3]) for i in seed['Seed']]

# create dataframe for win team and seed
winseeds = seed.rename(columns={'Team':'Wteam', 'numeric_seed':'win_seed'})
winseeds = winseeds[['Season','Wteam','win_seed']]

# create dataframe for loss team and seed
lossseeds = seed.rename(columns={'Team':'Lteam', 'numeric_seed':'loss_seed'})
lossseeds = lossseeds[['Season','Lteam','loss_seed']]

In [24]:
# merge seed for win team 
t_compact = pd.merge(left=t_compact, right=winseeds, how='left', on=['Season', 'Wteam'])
# merge seed for loss team
t_compact = pd.merge(left=t_compact, right=lossseeds, on=['Season', 'Lteam'])

# create new column for different seed between win team and loss team
t_compact['seed_diff'] = t_compact['win_seed'] - t_compact['loss_seed']

t_compact = t_compact.drop(['win_seed','loss_seed'], axis=1)

# Create dataset for feeding in the prediction model

In [25]:
# change column name
r_detail_mean_win = r_detail_mean.rename(columns={'team':'Wteam'})
# merge r_detail for win team
t_compact = pd.merge(left=t_compact, right=r_detail_mean_win, how='left', on=['Season', 'Wteam'])

# change column name
r_detail_mean_loss = r_detail_mean.rename(columns={'team':'Lteam'})
# merge r_detail for loss team
t_compact = pd.merge(left=t_compact, right=r_detail_mean_loss, how='left', on=['Season', 'Lteam'])


In [26]:
# create new columns for different between win team and loss team
t_compact['r_ast_diff'] = t_compact['ast_x'] - t_compact['ast_y']
t_compact['r_blk_diff'] = t_compact['blk_x'] - t_compact['blk_y']
t_compact['r_dr_diff'] = t_compact['dr_x'] - t_compact['dr_y']
t_compact['r_fga_diff'] = t_compact['fga_x'] - t_compact['fga_y']
t_compact['r_fga3_diff'] = t_compact['fga3_x'] - t_compact['fga3_y']
t_compact['r_fgm_diff'] = t_compact['fgm_x'] - t_compact['fgm_y']
t_compact['r_fgm3_diff'] = t_compact['fgm3_x'] - t_compact['fgm3_y']
t_compact['r_fta_diff'] = t_compact['fta_x'] - t_compact['fta_y']
t_compact['r_ftm_diff'] = t_compact['ftm_x'] - t_compact['ftm_y']
t_compact['r_pf_diff'] = t_compact['pf_x'] - t_compact['pf_y']
t_compact['r_score_diff'] = t_compact['score_x'] - t_compact['score_y']
t_compact['r_stl_diff'] = t_compact['stl_x'] - t_compact['stl_y']
t_compact['r_to_diff'] = t_compact['to_x'] - t_compact['to_y']
t_compact['r_fgp_diff'] = t_compact['fgp_x'] - t_compact['fgp_y']
t_compact['r_fgp3_diff'] = t_compact['fgp3_x'] - t_compact['fgp3_y']
t_compact['r_ftp_diff'] = t_compact['ftp_x'] - t_compact['ftp_y']

# drop columns
t_compact = t_compact.drop(['ast_x', 'blk_x', 'dr_x',
       'fga_x', 'fga3_x', 'fgm_x', 'fgm3_x', 'fta_x', 'ftm_x', 'pf_x',
       'score_x', 'stl_x', 'to_x', 'fgp_x', 'fgp3_x', 'ftp_x', 'ast_y',
       'blk_y', 'dr_y', 'fga_y', 'fga3_y', 'fgm_y', 'fgm3_y', 'fta_y', 'ftm_y',
       'pf_y', 'score_y', 'stl_y', 'to_y', 'fgp_y', 'fgp3_y', 'ftp_y'], axis=1)

t_compact = t_compact.dropna()

In [22]:
t_compact.head()

,Season,Wteam,Lteam,t_number_win_diff,seed_diff,r_ast_diff,r_blk_diff,r_dr_diff,r_fga_diff,r_fga3_diff,...,r_fgm3_diff,r_fta_diff,r_ftm_diff,r_pf_diff,r_score_diff,r_stl_diff,r_to_diff,r_fgp_diff,r_fgp3_diff,r_ftp_diff
1140,2003,1143,1301,0.0,-1,1.333333,-0.273563,2.345977,5.390805,-5.465517,...,-1.552874,-0.949425,-2.387356,-1.563218,2.082759,-1.214943,-0.027586,0.010234,0.025371,-0.089516
1147,2003,1281,1356,1.0,-5,0.175269,1.122581,3.436559,1.326882,5.134409,...,1.383871,-1.238710,-1.732258,-2.462366,-2.759140,-2.256989,1.112903,-0.029774,-0.035819,-0.049254
1156,2003,1231,1104,4.0,-3,1.705357,1.558036,1.696429,-2.241071,1.767857,...,1.174107,1.696429,1.267857,0.026786,1.245536,-2.169643,-1.785714,0.011055,0.018418,-0.000529
1159,2003,1268,1423,5.0,-5,4.226190,4.707143,3.616667,6.957143,-1.435714,...,-0.016667,3.371429,2.645238,0.566667,9.419048,1.692857,2.640476,0.001049,0.035317,0.013662
1170,2003,1163,1390,2.0,1,0.988172,4.378495,2.770968,4.167742,-3.687097,...,-0.868817,1.325806,0.867742,-0.116129,7.775269,0.417204,1.477419,0.029502,0.025274,-0.001630


In [27]:
# create train dataframe with only contain winning result
train = t_compact.drop(['Season','Wteam','Lteam'], axis=1)
train['result'] = 1

In [28]:
# create dataframe with only contain loss result
train_loss = t_compact.drop(['Season','Wteam','Lteam'], axis=1)

for i in train_loss.columns:
    train_loss[i] = -train_loss[i]

train_loss['result'] = 0

In [29]:
# concat winning part and loss part to create train dataset
train = pd.concat([train, train_loss])  #  (4100, 3)

In [26]:
train.head()

,t_number_win_diff,seed_diff,r_ast_diff,r_blk_diff,r_dr_diff,r_fga_diff,r_fga3_diff,r_fgm_diff,r_fgm3_diff,r_fta_diff,r_ftm_diff,r_pf_diff,r_score_diff,r_stl_diff,r_to_diff,r_fgp_diff,r_fgp3_diff,r_ftp_diff,result
1140,0.0,-1,1.333333,-0.273563,2.345977,5.390805,-5.465517,3.011494,-1.552874,-0.949425,-2.387356,-1.563218,2.082759,-1.214943,-0.027586,0.010234,0.025371,-0.089516,1
1147,1.0,-5,0.175269,1.122581,3.436559,1.326882,5.134409,-1.205376,1.383871,-1.238710,-1.732258,-2.462366,-2.759140,-2.256989,1.112903,-0.029774,-0.035819,-0.049254,1
1156,4.0,-3,1.705357,1.558036,1.696429,-2.241071,1.767857,-0.598214,1.174107,1.696429,1.267857,0.026786,1.245536,-2.169643,-1.785714,0.011055,0.018418,-0.000529,1
1159,5.0,-5,4.226190,4.707143,3.616667,6.957143,-1.435714,3.395238,-0.016667,3.371429,2.645238,0.566667,9.419048,1.692857,2.640476,0.001049,0.035317,0.013662,1
1170,2.0,1,0.988172,4.378495,2.770968,4.167742,-3.687097,3.888172,-0.868817,1.325806,0.867742,-0.116129,7.775269,0.417204,1.477419,0.029502,0.025274,-0.001630,1


In [30]:
X = train.drop(['result'], axis=1).values
y = train['result'].values

In [31]:
Xs = StandardScaler().fit_transform(X)

In [32]:
log = LogisticRegression()
cross_val_score(log, Xs, y, cv=5).mean()

0.72150537634408596

In [33]:
clf = svm.SVC()
clf.fit(Xs, y)
cross_val_score(clf, Xs, y, cv=5).mean()

0.72172043010752696

In [34]:
knn = KNeighborsClassifier()
knn.fit(Xs, y)
cross_val_score(knn, Xs, y, cv=5).mean()

0.66236559139784945

In [35]:
dtc = DecisionTreeClassifier(max_depth=100, random_state=0)
dtc.fit(Xs, y)
cross_val_score(dtc, Xs, y, cv=5).mean()

0.6253763440860215

In [36]:
bagger = BaggingClassifier(dtc)
cross_val_score(bagger, Xs, y, cv=5).mean()

0.67860215053763451

In [37]:
rfc = RandomForestClassifier(max_depth=200, n_estimators=200)
rfc.fit(Xs, y)
cross_val_score(rfc, Xs, y, cv=5).mean()

0.71139784946236562

In [38]:
gbc = GradientBoostingClassifier()
gbc.fit(Xs, y)
cross_val_score(gbc, Xs, y, cv=5).mean()

0.65505376344086019

# Make a prediction for each game of 2017 NCAA Men's basketball tournament

In [13]:
# load submission dataset
submission = pd.read_csv('SampleSubmission.csv').drop(['Pred'],axis=1)

In [14]:
# create column which contain left_team Id
submission['left_team'] = [int(i.split('_')[1]) for i in submission['Id']]

# create column which contain left_team Id
submission['right_team'] = [int(i.split('_')[2]) for i in submission['Id']]

In [39]:
# get t_win_games for 2017
tt_win_games = t_win_games[t_win_games['Season']==2017].drop(['Season'],axis=1)

# change column name
tt_win_games_left = tt_win_games.rename(columns={'Wteam':'left_team'})
# merge r_detail for win team
submission = pd.merge(left=submission, right=tt_win_games_left, how='left', on=['left_team'])

# change column name
tt_win_games_right = tt_win_games.rename(columns={'Wteam':'right_team'})
# merge r_detail for loss team
submission = pd.merge(left=submission, right=tt_win_games_right, how='left', on=['right_team'])

In [40]:
# fill the missong value with 0, I guess those team didn't play last year's t
submission = submission.fillna(0)

In [41]:
# create t_number_win_diff between left team and right team
submission['t_number_win_diff'] = submission['number_win_x'] - submission['number_win_y']
submission = submission.drop(['number_win_x','number_win_y'],axis=1)

In [42]:
# create dataframe contain 2017 r stat for each team
r_detail_mean_2017 = rwl_detail.groupby(['Season' ,'team']).mean().reset_index()

r_detail_mean_2017 = r_detail_mean_2017[r_detail_mean_2017['Season']==2017]
r_detail_mean_2017 = r_detail_mean_2017.drop(['Season'],axis=1)

In [43]:
# change column name
tr_detail_mean_2017_left = r_detail_mean_2017.rename(columns={'team':'left_team'})
# merge r_detail for win team
submission = pd.merge(left=submission, right=tr_detail_mean_2017_left, how='left', on=['left_team'])

# change column name
tr_detail_mean_2017_right = r_detail_mean_2017.rename(columns={'team':'right_team'})
# merge r_detail for loss team
submission = pd.merge(left=submission, right=tr_detail_mean_2017_right, how='left', on=['right_team'])

In [44]:
# create new columns for different between win team and loss team
submission['r_ast_diff'] = submission['ast_x'] - submission['ast_y']
submission['r_blk_diff'] = submission['blk_x'] - submission['blk_y']
submission['r_dr_diff'] = submission['dr_x'] - submission['dr_y']
submission['r_fga_diff'] = submission['fga_x'] - submission['fga_y']
submission['r_fga3_diff'] = submission['fga3_x'] - submission['fga3_y']
submission['r_fgm_diff'] = submission['fgm_x'] - submission['fgm_y']
submission['r_fgm3_diff'] = submission['fgm3_x'] - submission['fgm3_y']
submission['r_fta_diff'] = submission['fta_x'] - submission['fta_y']
submission['r_ftm_diff'] = submission['ftm_x'] - submission['ftm_y']
submission['r_pf_diff'] = submission['pf_x'] - submission['pf_y']
submission['r_score_diff'] = submission['score_x'] - submission['score_y']
submission['r_stl_diff'] = submission['stl_x'] - submission['stl_y']
submission['r_to_diff'] = submission['to_x'] - submission['to_y']
submission['r_fgp_diff'] = submission['fgp_x'] - submission['fgp_y']
submission['r_fgp3_diff'] = submission['fgp3_x'] - submission['fgp3_y']
submission['r_ftp_diff'] = submission['ftp_x'] - submission['ftp_y']

# drop columns
submission = submission.drop(['ast_x', 'blk_x', 'dr_x',
       'fga_x', 'fga3_x', 'fgm_x', 'fgm3_x', 'fta_x', 'ftm_x', 'pf_x',
       'score_x', 'stl_x', 'to_x', 'fgp_x', 'fgp3_x', 'ftp_x', 'ast_y',
       'blk_y', 'dr_y', 'fga_y', 'fga3_y', 'fgm_y', 'fgm3_y', 'fta_y', 'ftm_y',
       'pf_y', 'score_y', 'stl_y', 'to_y', 'fgp_y', 'fgp3_y', 'ftp_y'], axis=1)

In [45]:
# create dataframe only have season 2017
test_seed = seed[seed['Season']==2017]

# create dataframe for win team and seed
test_leftseeds = test_seed.rename(columns={'Team':'left_team', 'numeric_seed':'left_seed'})
test_leftseeds = test_leftseeds[['left_team','left_seed']]

# create dataframe for loss team and seed
test_rightseeds = test_seed.rename(columns={'Team':'right_team', 'numeric_seed':'right_seed'})
test_rightseeds = test_rightseeds[['right_team','right_seed']]

In [46]:
# merge seed for left team 
submission = pd.merge(left=submission, right=test_leftseeds, how='left', on=['left_team'])
# merge seed for loss team
submission = pd.merge(left=submission, right=test_rightseeds, how='left', on=['right_team'])

# create new column for different seed between win team and loss team
submission['seed_diff'] = submission['left_seed'] - submission['right_seed']

submission = submission.drop(['left_seed','right_seed'],axis=1)

In [46]:
submission.head(2)

,Id,left_team,right_team,t_number_win_diff,r_ast_diff,r_blk_diff,r_dr_diff,r_fga_diff,r_fga3_diff,r_fgm_diff,...,r_fta_diff,r_ftm_diff,r_pf_diff,r_score_diff,r_stl_diff,r_to_diff,r_fgp_diff,r_fgp3_diff,r_ftp_diff,seed_diff
0,2017_1112_1116,1112,1116,0.0,-0.294118,-1.500000,1.529412,-5.088235,-1.000000,-1.529412,...,-0.764706,-0.676471,-3.617647,-3.529412,-2.029412,-0.294118,0.015773,0.031094,-0.009186,-6
1,2017_1112_1124,1112,1124,0.0,-1.689753,-1.635674,1.535104,-0.137571,-1.768501,0.431689,...,3.685009,3.666034,-0.706831,4.651803,0.370968,-1.978178,0.009828,0.037155,0.069138,-1


In [47]:
testX = submission.drop(['Id','left_team','right_team'],axis=1).values

In [48]:
testXs = StandardScaler().fit_transform(testX)

In [53]:
rfc.predict(testXs)

array([1, 1, 0, ..., 1, 1, 1])

In [54]:
rfc.predict_proba(testXs)[:,1]

array([ 0.645,  0.56 ,  0.47 , ...,  0.705,  0.835,  0.855])

In [55]:
final_submission = submission[['Id']].copy()

In [56]:
final_submission['pred'] = rfc.predict_proba(testX)[:,1]

In [57]:
final_submission.head()

,Id,pred
0,2017_1112_1116,0.490
1,2017_1112_1124,0.550
2,2017_1112_1137,0.500
3,2017_1112_1139,0.475
4,2017_1112_1153,0.590
